_____________________
# ABOUT
_____________________

#### In this project we train a Transformer decoder type model with masked attention at Charater Level.  After training the model generates new lyrics for songs, given the initial sentence
#### The project is HEAVILY INSPIRED from Andrej Karpathy's MiniGPT implementation
#### The lisence info can be  found in the file : LISENCE
#### Lets begin

_____________________
# IMPORTS & SETUP
_____________________

In [ ]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

# Set seeds for all modules
from src.utils import set_seed, current_moment, predict_in_a_cool_way

set_seed(42)

import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

from src.dataset import CharDataset
from src.model import GPT, GPTConfig
from src.trainer import Trainer, TrainerConfig

block_size = 128 # Maximum context length allowed

_____________________
# DATASET
_____________________

In [ ]:
# DATA is stored at <data> folder, test/train ratio 1:9
# You can change <tiny> flag to True in order to do test runs on a very small subset of the data 

tiny = True        # Set this flag True for small data subset

if tiny:
    train_text = open('./data/train_tiny.txt', 'r').read() 
    test_text  = open('./data/test_tiny.txt' , 'r').read() 
else:
    train_text = open('./data/train.txt', 'r').read() 
    test_text  = open('./data/test.txt' , 'r').read() 

train_dataset = CharDataset(train_text, block_size) 
test_dataset = CharDataset(test_text, block_size) 

_____________________
# MODEL
Instatntiate the model. Feel free to change the number of layer and heads below
_____________________

In [ ]:
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size, n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

_____________________
# TRAINING

Now we train the model. Feel free to change the trainer configuration below according to your choice
_____________________

In [ ]:
# Training Configuration & trainer_config
trained_model_path = './models/'  + current_moment() + '_best.ckpt'         # The trained model is saved in <models> folder
trainer_config = TrainerConfig(max_epochs=5, batch_size=128, learning_rate=6e-4, lr_decay=True, warmup_tokens=512*20,       # Trainer configuration
                               final_tokens=2*len(train_dataset)*block_size, num_workers=4, ckpt_path=trained_model_path)

trainer = Trainer(model=model, train_dataset=train_dataset, test_dataset=test_dataset, config =trainer_config)              # Trainer object

In [ ]:
# Lets train the model
trainer.train()

_____________________
# INFERENCE for the above trained model
The best model checkpoint is saved at 'models' folder, correspontding to the specific date and time at which the model was made
_____________________

In [3]:
print(trained_model_path)

NameError: name 'trained_model_path' is not defined

In [ ]:
# Load the saved model
checkpoint = torch.load(trained_model_path)
model.load_state_dict(checkpoint)

In [ ]:
# Please enter the context for the lyric generation, you can also change the output length below
context = 'Tu hi meri shab hai subah hai tu hi din hai mera'

predict_in_a_cool_way(context=context, train_dataset=train_dataset, model=model, trainer=trainer, output_len=800)

_________________________________
# <center>THANK YOU</center>
_________________________________
